### Use the menu entry "Run &rarr; Run All Cells" to start this notebook.

In [ ]:
%pip install -q markdown ipywidgets==8.0.5 ipympl

import markdown, traceback, gettext
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.artist import Artist
from IPython.display import HTML
from ipywidgets import AppLayout, Layout
import ipywidgets as widgets

gettext.bindtextdomain('rungesphenomenon', 'translations')
gettext.textdomain('rungesphenomenon')
_ = gettext.gettext

out = widgets.Output()
mathFormulaOutput = widgets.Output()

# für interaktive matplotlib-Widgets in Jupyter notwendig:
# pip install ipympl
%matplotlib widget

# Example 1: (Square function)
example1_x = np.linspace(-2, 2, 150)
example1_y = example1_x**2

# Example 2: (Runge function)
example2_x = np.linspace(-2, 2, 150)
example2_y = 1 / (1 + 25*example2_x**2)

In [ ]:
class Interpolator:
    
    # max pixel distance to count as a vertex hit
    maxDistance = 15
    
    # the last observed toolbar mode
    lastToolBarMode = ""
    
    example1Plot = None
    example2Plot = None
    interpolatePlot = None
    
    title = _('Click to add points, tap "d" to delete them')
    
    def __init__(self, ax, line):
        if line.figure is None:
            raise RuntimeError('You must first add the polygon to a figure '
                               'or canvas before defining the interactor')
        self.ax = ax
        self.ax.set_title(self.title)
        
        self.line = line
        
        canvas = line.figure.canvas
        canvas.mpl_connect('draw_event', self.on_draw)
        canvas.mpl_connect('button_press_event', self.on_button_press)
        canvas.mpl_connect('key_press_event', self.on_key_press)
        self.canvas = canvas
        
    def get_index_under_point(self, event):
        """
        Return the index of the point closest to the event position or *None*
        if no point is within ``self.maxDistance`` to the event position.
        """
            
        # convert data to pixel dimensions
        xData = []
        xData.extend(self.line.get_xdata())
        
        if len(xData) == 0:
            # line is empty, nothing to delete...
            return None
        
        yData = []
        yData.extend(self.line.get_ydata())
        zippedList = list(zip(xData, yData))
        pixelData = ax.transData.transform(zippedList)

        # calculate distance to every point of the line
        xValues = np.asarray(pixelData[:, 0])
        yValues = np.asarray(pixelData[:, 1])
        distances = np.hypot(xValues - event.x, yValues - event.y)

        # return index of shortest distance (if valid)
        index = np.argmin(distances)
        if distances[index] > self.maxDistance:
            return None
        return index
    
    def add_point(self, event):
        if event.xdata == None or event.ydata == None:
            return

        # append new point to line
        newXdata = []
        newXdata.extend(self.line.get_xdata())
        newXdata.append(event.xdata)
        newYdata = []
        newYdata.extend(self.line.get_ydata())
        newYdata.append(event.ydata)

        # sort points by x-value
        zippedList = list(zip(newXdata, newYdata))        
        sortedList = sorted(zippedList, key = lambda x: x[0])

        # set new data
        newXdata, newYdata = list(zip(*sortedList))
        self.line.set_data(newXdata, newYdata)

        self.interpolate()

        if self.line.stale:
            self.canvas.draw_idle()        
    
    def on_draw(self, event):
        self.background = self.canvas.copy_from_bbox(self.ax.bbox)
        self.ax.draw_artist(self.line)
        
    def on_button_press(self, event):
        # only add points if no toolbar mode (e.g. "zoom rect" or "pan/zoom") is active
        toolBarMode = plt.get_current_fig_manager().toolbar.mode

        # if we have been in "zoom rect" or "pan/zoom" mode we have to click the figure again
        # to get the keyboard focus, don't add a point in this case
        if toolBarMode == '' and self.lastToolBarMode == '':
                self.add_point(event)

        self.lastToolBarMode = toolBarMode

    @out.capture(clear_output=True)
    def on_key_press(self, event):
        try:
            if not event.inaxes:
                return

            elif event.key == 'd':
                index = self.get_index_under_point(event)
                if index is not None:
                    # remove point from line
                    newXdata = []
                    newXdata.extend(self.line.get_xdata())
                    del newXdata[index]
                    newYdata = []
                    newYdata.extend(self.line.get_ydata())
                    del newYdata[index]
                    self.line.set_data(newXdata, newYdata)

                self.interpolate()

            if self.line.stale:
                self.canvas.draw_idle()
        except:
            traceback.print_exc()
            
    def interpolate(self):
        xData = self.line.get_xdata()
        
        if len(xData) == 0:
            plot_x = []
            plot_y = []
        else:
            # polynomial interpolation
            vanderMondeMatrix = np.vander(xData, len(xData))
            yData = self.line.get_ydata()
            coefficients = np.linalg.solve(vanderMondeMatrix, yData)
            polynomial = np.poly1d(coefficients)
            plot_x = np.linspace(xData[0], xData[-1], 150)
            plot_y = polynomial(plot_x)
            
            # output math formula
            coefficients = list(reversed(polynomial.coefficients))
            markdownString = ""
            order = 0
            for i in range(0, len(coefficients)):
                coefficient = coefficients[i]
                if i == 0:
                    markdownString = "{:.4f}".format(coefficient)
                else:
                    newTerm = "{:.4f}".format(coefficient) + "x"
                    if order != 1:
                        newTerm += "^{" + str(order) + "}"
                    if coefficients[i - 1] > 0:
                        newTerm += "+ "
                    markdownString = newTerm + markdownString
                order += 1            
            mathFormulaOutput.append_display_data(HTML(markdown.markdown("$" + markdownString + "$")))

        # update plot
        if self.interpolatePlot != None:
            self.interpolatePlot.pop(0).remove()
        self.interpolatePlot = self.ax.plot(plot_x, plot_y, color='red')
        
    @out.capture(clear_output=True)
    def addExample1(self):
        try:
            if self.example1Plot != None and len(self.example1Plot) > 0:
                self.example1Plot.pop(0).remove()
            self.example1Plot = self.ax.plot(example1_x, example1_y, color='green')
            self.canvas.draw_idle()
        except:
            traceback.print_exc()

    @out.capture(clear_output=True)
    def addExample2(self):
        try:
            if self.example2Plot != None and len(self.example2Plot) > 0:
                self.example2Plot.pop(0).remove()
            self.example2Plot = self.ax.plot(example2_x, example2_y, color='blue')
            self.canvas.draw_idle()
        except:
            traceback.print_exc()
            
    @out.capture(clear_output=True)
    def restart(self):
        try:
            self.line.set_data([], [])
            if self.interpolatePlot != None:
                self.interpolatePlot.pop(0).remove()
            if self.example1Plot != None and len(self.example1Plot) > 0:
                self.example1Plot.pop(0).remove()
            if self.example2Plot != None and len(self.example2Plot) > 0:
                self.example2Plot.pop(0).remove()
            self.interpolatePlot = self.ax.plot([], [])
            self.canvas.draw_idle()
            mathFormulaOutput.clear_output()
        except:
            traceback.print_exc()

def example1_button_clicked(b):
    interpolator.addExample1()

def example2_button_clicked(b):
    interpolator.addExample2()

def restart_button_clicked(b):
    interpolator.restart()
            
def clean_button_clicked(b):
    mathFormulaOutput.clear_output()
        
if __name__ == '__main__':
    
    xValues = []
    yValues = []
    
    line = Line2D(xValues, yValues, marker='o', markerfacecolor='red', linestyle='None', animated=True)

    with out:
        plt.ioff() # turn off interactive mode so figure doesn't show
        fig, ax = plt.subplots()
        plt.ion() # figure still doesn't show
    ax.add_line(line)
    global interpolator
    interpolator = Interpolator(ax, line)

    ax.set_xlim((-2, 2))
    ax.set_ylim((-1, 1.5))
        
    out.clear_output()
    mathFormulaOutput.clear_output()
    
    headerOutput = widgets.Output()
    with headerOutput:
        display(HTML(_('<h1>Runge&apos;s phenomenon</h1> \
            Runge&apos;s phenomenon shows the problem that going to higher degrees does not always improve accuracy in polynomial interpolation.<br> \
            See &rarr; <a href="https://en.wikipedia.org/wiki/Runge%27s_phenomenon" target="_blank">this Wikipedia article</a> &larr; for more details.<br><hr>')))
    headerOutput.layout = Layout(margin='0 0 20px 0')
    
    addExample1Button = widgets.Button(description=_("Add Quadratic Function"))
    addExample1Button.on_click(example1_button_clicked)

    addExample2Button = widgets.Button(description=_("Add Runge Function"))
    addExample2Button.on_click(example2_button_clicked)
    
    restartButton = widgets.Button(description=_("Restart"))
    restartButton.on_click(restart_button_clicked)

    addExample1Button.layout = Layout(flex='0 0 auto', width="fit-content")
    addExample2Button.layout = Layout(flex='0 0 auto', width="fit-content")
    restartButton.layout = Layout(flex='0 0 auto', width="fit-content")
    buttonHBox = widgets.HBox([addExample1Button, addExample2Button, restartButton])
    
    leftVBox = widgets.VBox([buttonHBox, fig.canvas])
    leftVBox.layout = Layout(flex='0 0 auto', width='fit-content')

    cleanButton = widgets.Button(description=_("Clear output"))
    cleanButton.on_click(clean_button_clicked)
    
    cleanButton.layout = Layout(flex='0 0 auto', width="fit-content")
    mathFormulaOutput.layout = Layout(border = "1px solid black")
    
    rightVBox = widgets.VBox([cleanButton, mathFormulaOutput])
    rightVBox.layout = Layout(flex='1 1 auto', width='fit-content')
    
    appHBox = widgets.HBox([leftVBox, rightVBox])
    
    display(widgets.VBox([headerOutput, appHBox]))

In [ ]:
out